In [2]:
import pandas as pd

In [3]:
df_matches = pd.read_csv("./matches/processed_ds/matches.csv", parse_dates=['date'])
df_ratings = pd.read_csv("./ratings/dataset/ratings.csv")
df_xg = pd.read_csv("./xg/processed_ds/xg.csv")
df_stats = pd.read_csv("./match_stats/processed_ds/match_stats.csv")

# Making the team names consitent across the datasets

In [4]:
pd.melt(frame=df_stats, id_vars='date', value_vars=['home', 'away'])['value'].value_counts()

Real Sociedad          214
Atletico Madrid        214
Getafe                 214
Villarreal             213
Athletic Bilbao        213
Valencia               213
Barcelona              213
Sevilla                212
Real Madrid            212
Celta Vigo             211
Real Betis             209
Levante                190
Alaves                 190
Espanyol               174
Eibar                  152
Real Valladolid        137
Osasuna                137
Granada                114
Leganes                113
Girona                  99
Elche                   99
Real Mallorca           99
Rayo Vallecano          99
Cadiz                   99
SD Huesca               76
Deportivo La Coruna     38
Malaga                  38
Las Palmas              38
Almeria                 24
Name: value, dtype: int64